The orginal source of the code:
https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-mlflow/sagemaker_deployment_mlflow.html

Install libraries

In [1]:
!pip install mlflow==2.13.2 sagemaker-mlflow==0.1.0

  Using cached mlflow-2.13.2-py3-none-any.whl.metadata (29 kB)
  Using cached sagemaker_mlflow-0.1.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached alembic-1.13.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached graphene-3.3-py2.py3-none-any.whl.metadata (7.7 kB)
  Using cached opentelemetry_api-1.25.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_sdk-1.25.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl.metadata (559 bytes)
  Using cached gunicorn-22.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached Mako-1.3.5-py3-none-any.whl.metadata (2.9 kB)
  Using cached graphql_core-3.2.3-py3-none-any.whl.metadata (10 kB)
  Using cached graphql_relay-3.2.0-py3-none-any.whl.metadata (12 kB)
  Using cached aniso8601-9.0.1-py2.py3-none-any.whl.metadata (23 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached opentelemetry_semantic_conventions-0.46b0-py3-none-any.whl.metadata (2.3 kB)
Usi

In [9]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.serve import SchemaBuilder
from sagemaker.serve import ModelBuilder
from sagemaker.serve.mode.function_pointers import Mode
import mlflow
from mlflow import MlflowClient
import boto3
import numpy as np
import pandas as pd
import os
from sklearn import datasets

SETTINGS

In [16]:
tracking_server_arn = "arn:aws:sagemaker:eu-west-1:211125740051:mlflow-tracking-server/Sample-server"
bucket_name = 'sagemaker-bucket-ds'
project_path_s3 = "iris-v3"

BASIC OBJECTS

In [5]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name
s3_client = boto3.client("s3")

CREATE FOLDERS

In [4]:
os.makedirs("./data", exist_ok=True)
os.makedirs("./training_code", exist_ok=True)

PREPARE DATA

In [11]:
iris = datasets.load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df["class"] = pd.Series(iris.target)
df = df[[list(df.columns)[-1]] + list(df.columns)[:-1]] # Reorder target as the first column
df.columns = df.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
iris = df.to_numpy()
np.savetxt('./data/iris.csv', iris, delimiter=',', fmt='%1.1f, %1.3f, %1.3f, %1.3f, %1.3f')

CREATE PATHS

In [32]:
data_path_s3 = os.path.join('s3://', bucket_name, project_path_s3, 'data') 
data_path_s3

's3://sagemaker-bucket-ds/iris-v3/data'

COPY DATA TO S3

In [21]:
!aws s3 cp data {data_path_s3} --recursive

upload: data/iris.csv to s3://sagemaker-bucket-ds/iris-v3/data/iris.csv


CREATE TRAINING ENTRY POINT

In [55]:
%%writefile training_code/train.py

from __future__ import print_function

import argparse
import joblib
import os
import pandas as pd

from sklearn import tree

import mlflow

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.
    #parser.add_argument('--max_leaf_nodes', type=int, default=-1)

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    #parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    #parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    #parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])

    #args = parser.parse_args()
    max_leaf_nodes = 30
    output_data_dir = os.environ['SM_OUTPUT_DATA_DIR']
    model_dir = os.environ['SM_MODEL_DIR']
    train_dir = os.environ['SM_CHANNEL_TRAIN']

    # Take the set of files and read them all into a single pandas dataframe
    input_files = [ os.path.join(train_dir, file) for file in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, file))]
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(train_dir, "train"))
    raw_data = [ pd.read_csv(file, header=None, engine="python") for file in input_files ]
    train_data = pd.concat(raw_data)

    # Set the Tracking Server URI using the ARN of the Tracking Server you created
    mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_ARN'])
    
    # Enable autologging in MLflow
    mlflow.autolog()

    # labels are in the first column
    train_y = train_data.iloc[:, 0]
    train_X = train_data.iloc[:, 1:]  

    # Now use scikit-learn's decision tree classifier to train the model.
    clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
    clf = clf.fit(train_X, train_y)

    # Print the coefficients of the trained classifier, and save the coefficients
    joblib.dump(clf, os.path.join(model_dir, "model.joblib"))
    
    # Register the model with MLflow
    run_id = mlflow.last_active_run().info.run_id
    artifact_path = "model"
    model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=run_id, artifact_path=artifact_path)
    model_details = mlflow.register_model(model_uri=model_uri, name="sm-job-experiment-model")

Overwriting training_code/train.py


CREATE REQUIREMENTS

In [24]:
%%writefile training_code/requirements.txt
mlflow==2.13.2
sagemaker-mlflow==0.1.0
cloudpickle==2.2.1 # Required for Sagemaker Python SDK

Writing training_code/requirements.txt


CREATE ESTIMATOR

In [56]:
sklearn = SKLearn(
    entry_point='train.py',
    source_dir='training_code',
    framework_version='1.2-1',
    instance_type='ml.m5.large',
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={'max_leaf_nodes': 30},
    keep_alive_period_in_seconds=3600,
    environment={
        'MLFLOW_TRACKING_ARN': tracking_server_arn
    },
    base_job_name='iris-v3'
)

FIT THE ESTIMATOR

In [57]:
sklearn.fit({"train": data_path_s3})

INFO:sagemaker:Creating training-job with name: iris-v3-2024-06-26-14-42-25-419


2024-06-26 14:42:25 Starting - Starting the training job...
2024-06-26 14:42:45 Starting - Preparing the instances for training...
2024-06-26 14:43:15 Downloading - Downloading input data...
2024-06-26 14:43:43 Downloading - Downloading the training image......
2024-06-26 14:44:44 Training - Training image download completed. Training in progress.2024-06-26 14:44:48,336 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-06-26 14:44:48,340 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-26 14:44:48,343 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-26 14:44:48,362 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-06-26 14:44:48,624 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2

In [13]:
mlflow.set_tracking_uri(tracking_server_arn)
client = MlflowClient()
registered_model = client.get_registered_model(name="sm-job-experiment-model")
source_path = registered_model.latest_versions[0].source

In [14]:
# Artifact URI of the model
source_path

's3://sagemaker-bucket-ds/ML_FLOW/0/5f5d29fc7fb8472094ee148ce7127d26/artifacts/model'

In [15]:
sklearn_input = np.array([1.0, 2.0, 3.0, 4.0]).reshape(1, -1)
sklearn_output = 1
sklearn_schema_builder = SchemaBuilder(
    sample_input=sklearn_input,
    sample_output=sklearn_output,
)

In [16]:
# Create model builder with the schema builder.
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    schema_builder=sklearn_schema_builder,
    role_arn=role,
    model_metadata={
        "MLFLOW_MODEL_PATH": source_path
    }
)

In [17]:
built_model = model_builder.build()

ModelBuilder: WARNING:     Support of MLflow format models is experimental and is not intended for production at this moment.


ModelBuilder: INFO:     Auto-detected deployment flavor is sklearn
ModelBuilder: INFO:     Auto-detected framework version is 1.2.1
ModelBuilder: INFO:     Auto detected 141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3. Proceeding with the the deployment.
ModelBuilder: INFO:     Skipping auto detection as the image uri is provided 141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3
ModelBuilder: DEBUG:     Packaging provided requirements.txt from /tmp/sagemaker/model-builder/a3edc70e33ab11efa8fce600116ddf77/requirements.txt


ModelBuilder: DEBUG:     Uploading the model resources to bucket=sagemaker-eu-west-1-211125740051, key_prefix=sagemaker-scikit-learn-2024-06-26-11-15-27-566.
Uploading model artifacts: 100%|████████████████████████| 11950/11950 [00:00<00:00, 18148.59bytes/s]
ModelBuilder: DEBUG:     Model resources uploaded to: s3://sagemaker-eu-west-1-211125740051/sagemaker-scikit-learn-2024-06-26-11-15-27-566/serve.tar.gz


In [18]:
predictor = built_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-06-26-11-16-19-747
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-06-26-11-16-20-351
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2024-06-26-11-16-20-351


-----------!

ModelBuilder: DEBUG:     ModelBuilder metrics emitted.


In [19]:
predictor.predict(sklearn_input)

[2.0]

In [21]:
sagemaker_session.delete_endpoint(endpoint_name=built_model.endpoint_name)

INFO:sagemaker:Deleting endpoint with name: sagemaker-scikit-learn-2024-06-26-11-16-20-351
